This is a notebook for the Week 4 of the Developer Challenge: https://blogs.sap.com/2023/03/08/sap-community-developer-challenge-eda-with-sap-hana-and-python/

Some code might be incomplete or incorrect. It is your task to fix it and execute the whole notebook receiving correct results.

Submit the link to your solution: https://groups.community.sap.com/t5/application-development/submissions-for-quot-eda-with-sap-hana-quot-developer-challenge/m-p/227591/highlight/true#M1208

## Initialize the notebook

Import required packages

In [ ]:
from hanaml import dataframe as hdf
import pandas as pd

Connect to SAP HANA database instance with a database user, same as during Week 2!

In [ ]:
myconn=hdf.ConnectionContext(userkey='DevChallenger')
print("SAP HANA DB version: ", myconn.hana_version())

In [ ]:
print(myconn.sql("SELECT NOW() FROM DUMMY").collect().CURRENT_TIMESTAMP[0])

## Check tables from Week 2 are available for this week's exercises

You should see these 3 tables listed: `BIKES`, `MONTHS`, `SEASONS`. If not, then please complete the Week 2 first.

In [ ]:
myconn.get_tables()

During Week 2 you should have created HANA DataFrame `bikes_hdf` at the moment of loading a CSV file. As the table with data already existed in your database, a week ago you used [the `table()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.ConnectionContext.table) to create the HANA DataFrame as-is.

This week use once again [a `select()` method](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.dataframe.html#hana_ml.dataframe.DataFrame.select) to add a calculated column `CalDay`, but as well to rename and rearange all other columns to make them easier to read and interpret.

In [ ]:
bikes_hdf=myconn.table('BIKES').select(('"days_since_2011"', "Days_Counter"),
                                        ('add_days(\'2011-01-01\',"days_since_2011")', "CalDay"),
                                        ('"yr"', "Year"),
                                        ('"season"', "Season"),
                                        ('"mnth"', "Month"),
                                        ('"weekday"', "Weekday"),
                                        ('"workingday"', "Is_WorkingDay"),
                                        ('"holiday"', "Is_Holiday"),
                                        ('"weathersit"', "Weather"),
                                        ('"temp"', "Temperature"),
                                        ('hum', "Humidity"),
                                        ('"windspeed"', "Wind_Speed"),
                                        ('"cnt"', "Bikes_Rented")
                                       )

Execute the cell below as is with a question mark at the end `bikes_hdf.select?` to [see the help for the method](https://ipython.readthedocs.io/en/latest/interactive/python-ipython-diff.html#accessing-help).

In [ ]:
bikes_hdf.select?

In [ ]:
bikes_hdf.select_statement

In [ ]:
bikes_hdf.head(5).collect().set_index('days_since_2011')

In [ ]:
bikes_hdf.count()

### Understand cells output

The [default output behavior](https://ipython.readthedocs.io/en/stable/config/options/terminal.html#configtrait-InteractiveShell.ast_node_interactivity) in a Jupyter notebook (with IPython kernel) is to show the evaluation result for the last expression in the cell.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity.default_value

That's why by default results are displayed when running a cell containing `bikes_hdf.count()` or `bikes_hdf.head(5).collect().set_index('Days_Counter')` without calling `display()`, like
```
display(bikes_hdf.head(5).collect().set_index('Days_Counter'))
```
or
```
display(bikes_hdf.count())
```

# 1. Data Visulaization using Pandas Plots

As Pandas dataframes store results of HANA DataFrame's `collect()` execution, data from these results can be visulazied using Pandas' [built-in support for data visualization through charts with matplotlib](https://pandas.pydata.org/docs/ecosystem.html#ecosystem-visualization).

In [ ]:
from matplotlib import pyplot as plt

To get consistent output across different clients you might use (like Jupyter, VS Code, IPython etc) execute [the magic `%matplotlib inline`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib) to have visualizations displayed within notebook's outputs.

In [ ]:
%matplotlib inline

As stayed at [the `matplotlib-inline` project's page](https://github.com/ipython/matplotlib-inline#usage):

> _Note that in current versions of JupyterLab and Jupyter Notebook, the explicit use of the `%matplotlib inline` directive is not needed anymore, though other third-party clients may still require it._

If you are using JupyterLab, then `plt.show()` might not be necessarily at the end of each cell as it is called by default in that client as described in https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.show.html#matplotlib-pyplot-show (or can be seen executing `plt.show?` in a notebook's cell):

> **Auto-show in jupyter notebooks**
>
> _The jupyter backends (activated via `%matplotlib inline`, `%matplotlib notebook`, or `%matplotlib widget`), call `show()` at the end of every cell by default. Thus, you usually don't have to call it explicitly there._

Execute the cell below as-is (with a question mark at the end) `plt.show?` [see the help for the method](https://ipython.readthedocs.io/en/latest/interactive/python-ipython-diff.html#accessing-help).

In [ ]:
plt.show?

To be consistent with clients other than Jupyter that you might use here, `plt.show()` is included at the end of each cell with data visualization.

## Foundations of plotting with Matplotlib

The [basic `plot()` method's](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html#pandas-dataframe-plot) execution plots line charts for all dataframe's numeric columns into a single [axes](https://matplotlib.org/stable/api/axes_api.html) on a single [figure](https://matplotlib.org/stable/api/figure_api.html).

In [ ]:
bikes_hdf.collect().plot()
plt.show()

A lot is going on on this figure, but there is very little to understand on the first sight. 

Let's unpack it to understnd and to improve later on.
1. The horizontal axis X has a scale equal to the number of records in a dataframe used, in this case the max value on the axis is equal to `bikes_hdf.count()` or **731**.
2. The vertical axis Y has a scale to accomodate all values from all numeric columns. In this case `Bikes_Rented` has values between 0 and **8000+**. It makes lines ploted for other columns, like `Humidity` with the scale between 0 and 100, unreadable.
3. Each column's values are ploted using a separate color. These colors are not random, but are coming from a default color cycler stored with a key `'axes.prop_cycle'` in [Runtime Configuration Parametes `rcParams`](https://matplotlib.org/stable/api/matplotlib_configuration_api.html#default-values-and-styling).

#### Understand the default color cycler

Code cells below will print hex codes from the default color cycler and display corresponding colors as a pie plot. Yes, the author of this notebook does like pie plots :-)

In [ ]:
prop_cycle_default = plt.rcParams['axes.prop_cycle']

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors_df = pd.Series(data=[1]*len(colors), index=colors)
display(colors_df)

In [ ]:
colors_df.plot(kind='pie', legend=True, figsize=(4,4))
plt.show()

The default order in Matplotlib's pie charts is counterclockwise starting from "3 o'clock" (or the "right side").

#### Change color cycler to follow SAP Fiori branding

Update the default cycler in this [kernel process](https://ipython.readthedocs.io/en/latest/development/how_ipython_works.html#the-ipython-kernel) to use secondary (_accent_) colors of the Belize theme from SAP Fiori: https://experience.sap.com/fiori-design-web/colors/#accent-colors

In [ ]:
from cycler import cycler 
sap_fiori_belize_cycler = cycler(
    color=['#0092D1', '#E09D00', '#1A9898', '#E6600D', '#925ACE', '#759421', '#678BC7', '#C14646', '#647987', '#AB218E']
)

plt.rc('axes', prop_cycle=sapgui_belize_cycler)

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors_df = pd.Series(data=[1]*len(colors), index=colors)

colors_df.plot(kind='pie', legend=True, figsize=4,4)
plt.show()

## Plot something meaningful

This time:
1. Plot data for a number of rented bikes daily and a temperature. Please note, that first two colors (with indexes `0` and `1`) from the new color cycler should now be used.
2. Set X axis to dates `x='CalDay'`.
3. Use a separate ("`secondary_y`") axis for a temperature.
4. Set bigger size for this figure: `figsize=(18, 6)`.

In [ ]:
(bikes_hdf
 .select('CalDay', 'Bikes_Rented', 'Temperature').collect()
 .plot(x='CalDay', secondary_x=['Temperature'], figsize=(18, 6))
)
plt.show()

You should be able to see:
1. seasonal cycles: temperatures and rentals are higher during summer, and lower during winter, but...
2. ...rental is somewhat dropping when a temperature rises above ~25 degrees Celcius.
3. There were less bicycles rented during the first summer, but that might be caused by a lower supply, i.e. a number of bikes provided during the first season. 

Set this figure size -- **18x6** inches -- as a new default for new figures in this notebook.

In [ ]:
figsize_default=plt.rcParams["figure.figsize"]
print("Previous default figure size was : ", figsize_default)
plt.rcParams["figure.figsize"] = (18, 6)
print("Current default figure size is   : ", plt.rcParams["figure.figsize"])

Now, let's plot humidity and wind speed, but using next colors -- with indexes `2` and `3` -- from the current color cycler. 

A `"Cn"` color spec, i.e. `"C"` followed by a number, which is an index into the default property cycle starting from 0, as described in  https://matplotlib.org/stable/gallery/color/color_demo.html#color-demo.

In [ ]:
(bikes_hdf
 .select('CalDay', 'Humidity', 'Wind_Speed').collect()
 .plot(x='CalDay', secondary_y=['Humidity'], color=('c2','c3'))
)
plt.show()

## Scaterplots

Check correlation between a temperature and a number of bikes rented using [a `plot.scatter` Pandas dataframe method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.scatter.html#pandas-dataframe-plot-scatter).

In [ ]:
(bikes_hdf
 .select('Bikes_Rented', 'Humidity', 'Temperature').collect()
 .plot.scatter(y='Bikes_Rented', x='Temperature')
)
plt.show()

You might notice a tendency that a number of rented bikes is growing with the temperature up to about +25C, but then it is becoming too hot and a number of bikes rented is dropping.

Add color coding for humidity values.

In [ ]:
(bikes_hdf
 .select('Bikes_Rented', 'Temperature', 'Humidity').collect()
 .plot.scater(y='Bikes_Rented', x='Temperature', c='Humidity')
)
plt.show()

You might notice that:
1. Colors for different values of the same column are not following the same color cycle as it was for different columns on charts above. Now a colorbar on the chart uses a colormap, by default it is `viridis`.
2. For the same temperature measures numbers of bikes rented have a tendency to be on a lower side for the higher humidity.

Now Matplotlib allows to use non-numberic columns for color-coding, but they must be converted to Pandas's [categoricals](https://pandas.pydata.org/docs/user_guide/categorical.html#categorical-data).

Replace color coding to use a `Season` column. To avoid default monochrome colormap used for categorical data, explicitely specify a [colormap](https://pandas.pydata.org/docs/user_guide/visualization.html#colormaps) to be `viridis`.

In [ ]:
(bikes_hdf
 .select('Bikes_Rented', 'Temperature', 'Season').collect().astype({"Season": 'category'})
 .plot.scatter(y='Bikes_Rented', x='Temperature', c='Season', 
               colormap='viridis' #try to comment this like to see the difference
              )
)
plt.show()

Now do he same for a `Year` column.

In [ ]:
(bikes_hdf
 .select('Bikes_Rented', 'Temperature', 'Year').collect().astype({"Year": 'category'})
 .plot.scatter(y='Bikes_Rented', x='Temperature', c='Year', colormap='Viridis')
)
plt.show()

## Plot categorical data

When analyzing categorical (not numerical columns) we usually analyze their values properties.

Visualize how often each weather situation occured in the HANA DataFrame.

In [ ]:
weather_df=bikes_hdf.value_counts(['Weather']).collect().set_index('VALUES')
display(weather_df)

Data scientists love to hate pie charts, so let's use them first 🤓

In [ ]:
(weather_df
 .plot.pie(y = 'NUM_Weather')
)
plt.show()

There are a few good practices when using pie charts:
1. Avoid using them for more than 3-5 categories.
1. Segments should be sorted from the biggest to the smallest.
2. Percentage values should be displayed to avoid guessing them by comparing angles of segments.
3. A human eye usually starts from 12 o'clock (the top of a circle) and then goes clock-wise.
4. Avoid unnecessary information, like having a legend on a figure.

In [ ]:
(weather_df
 .sort_values('NUM_Weather', ascending=False)
 .plot.pie(y = 'NUM_Weather', legend=False, 
        startangle=90, counterclock=False,
        autopct='%.2f', pctdistance=0.75)
)
plt.show()

As mentioned, data scientists love to hate pie charts, and many would say that [bar charts](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html#pandas.DataFrame.plot.bar) should be used instead of pies.

In [ ]:
weather_df.plot.bar()
plt.show()

Some good practices apply to bar/column charts as well:
1. Column charts should be used when category is related to the time, otherwise [horizontal bar charts](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.barh.html#pandas-dataframe-plot-barh) should be used.
2. Bars should be sorted either accordingly to their values or alphabetically by their names.
3. Value labels should be displayed to avoid guessing their values from a scale.

In [ ]:
(weather_df
 .sort_values('NUM_Weather', ascending=False)
 .plot.barh(legend=False)
)

#Using text annotations to calculate positions and place values next to their corresponding bars
for i in range(len(weather_df)):
    plt.text(y = i, 
             x = weather_df.sort_values('NUM_Weather', ascending=False).NUM_Weather[i]+4, # x position
             s = weather_df.sort_values('NUM_Weather', ascending=False).NUM_Weather[i],   # text to display
             ha = 'left')
plt.show()

# 2. Data Visulaization using `hana-ml` EDA

So far you have created plots for data stored in a Pandas dataframe after being selected from a HANA DataFrame.

Python Machine Learning Client for SAP HANA provides [the `hana_ml.visualizers` package](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.visualizers.html#hana-ml-visualizers-package) with many pre-arranged specialized statistics visualizations that push down calculations to SAP HANA database and then visualize results in a client using either Matplotlib (default) or Plotly (experiemental).

Most of capabilities from that package are not available in a trial environment (as of March 2023), so let's look at just a few example that are available in a trial.

In [ ]:
from hana_ml.visualizers.eda import EDAVisualizer

[EDAVisualizer](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.visualizers.html#hana_ml.visualizers.eda.EDAVisualizer) is `hana-ml`'s class for all EDA visualizations, including for example [pie plot](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.visualizers.html#hana_ml.visualizers.eda.EDAVisualizer.pie_plot) or [bar plot](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2022_4_QRC/en-US/hana_ml.visualizers.html#hana_ml.visualizers.eda.EDAVisualizer.bar_plot).

For most of the plotting methods it returns Pandas dataframe with the data used in the plot as a second returned value (`df` in examples below).

In [ ]:
_, df = EDAVisualizer().pie_plot(data=bikes_hdf, column='Wether',
                                 legend=False, explode=0, 
                                 startangle=90, counter_clock=False)
plt.show()

In [ ]:
display(df)

In [ ]:
_, df = EDAvisualizer().bar_plot(data_hdf=bikes, column='Weather', aggregation={'Weather':'count'})
plt.show()

In [ ]:
display(df)

# And that's it for now!

Thank you for joining the challenge, we hope you learned something new.

Please share your feedback and thoughts what else you would like to try in the comments to the https://blogs.sap.com/2023/03/08/sap-community-developer-challenge-eda-with-sap-hana-and-python/